In [1]:
import openeo
import geopandas as gpd
import leafmap
import shapely
from datetime import datetime, timedelta
import os 
from query import get_matching_s2_cdse

/home/vpremier/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## Authentication
- to the Copernicus Data Space Ecosystem https://openeo.dataspace.copernicus.eu/
- Vito backend https://openeo-dev.vito.be

In [2]:
eoconn = openeo.connect('https://openeo.dataspace.copernicus.eu/', auto_validate=False)
eoconn.authenticate_oidc()

eoconn.describe_account()

Authenticated using refresh token.


{'info': {'oidc_userinfo': {'email': 'valentina.premier@eurac.edu',
   'email_verified': True,
   'family_name': 'Premier',
   'given_name': 'Valentina',
   'name': 'Valentina Premier',
   'preferred_username': 'valentina.premier@eurac.edu',
   'sub': '1aa67f75-e178-495e-a131-9f7dffaaae18'}},
 'name': 'Valentina Premier',
 'user_id': '1aa67f75-e178-495e-a131-9f7dffaaae18'}

## Show the study area and get the bounding box
- Read a shapefile with the AOI
- Get the basin outline
- Show on the map

In [3]:
shp_path = r'/home/vpremier/OEMC/test_Kathi/SierraNevada/SierraNevada.shp'

catchment_outline = gpd.read_file(shp_path)
bbox = catchment_outline.bounds.iloc[0]

center = (float(catchment_outline.centroid.y), 
          float(catchment_outline.centroid.x))
m = leafmap.Map(center=center, zoom=1)
m.add_vector(shp_path, layer_name="catchment")
m

Map(center=[37.08384948249926, -3.1703046934694843], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Set some basic information:
- time range
- cloud probabilty
- credentials for CDSE
- bands needed
- target extent, resolution and reference system

In [4]:
startdate = '2023-08-02'
enddate = '2023-08-15'

cloud_prob = 100

username = "valentina.premier@eurac.edu"
psw = "Openeo_290691"

bands = ['B01','B02','B03','B04','B05','B06','B07','B08',
         'B8A','B09','B10','B11','B12']

outdir = '/home/vpremier/OEMC/test_Kathi'

west=631810.
south=5167710.
east=655790.
north=5184190.
area = shapely.geometry.box(west, south, east, north) 
res = 20.
crs = 32632

Query the data through CDSE and get the available dates

In [5]:
s2List = get_matching_s2_cdse(startdate, enddate, username, psw, shp = shp_path,
                         max_cc = cloud_prob, tile = None, filter_date = True)
s2List

Found 17 Sentinel-2 scenes from 2023-08-02 to 2023-08-15 with maximum cloud coverage 100%


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,commonName
0,application/octet-stream,12d8c1c8-5b37-43d8-999c-f92a9aacd0c8,S2A_MSIL1C_20230803T104631_N0509_R051_T30SVF_2...,application/octet-stream,117612897,2023-08-03T19:32:41.400000Z,2023-08-03T19:40:56.601801Z,2023-08-03T19:41:01.085805Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/08/03/S2A_MSIL...,"[{'Value': 'b0ed57ed68efdd60682b0c8fc2f23ba2',...","{'Start': '2023-08-03T10:46:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-3.1923828 36.05...,"{'type': 'Polygon', 'coordinates': [[[-3.19238...",S2A_MSIL1C_20230803T104631_R051_T30SVF
1,application/octet-stream,86a4da95-73bc-4853-8337-80c1d6afbaf8,S2A_MSIL1C_20230803T104631_N0509_R051_T30SWF_2...,application/octet-stream,741214677,2023-08-03T19:21:40.125000Z,2023-08-03T19:30:29.976824Z,2023-08-03T19:30:40.890650Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/08/03/S2A_MSIL...,"[{'Value': '6b0da7d2d2b665b3dc197fa0edf0cadf',...","{'Start': '2023-08-03T10:46:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-3.0002136 36.73...,"{'type': 'Polygon', 'coordinates': [[[-3.00021...",S2A_MSIL1C_20230803T104631_R051_T30SWF
2,application/octet-stream,f0593297-e58d-4e43-b1da-771cbc0a9449,S2A_MSIL1C_20230803T104631_N0509_R051_T30SWG_2...,application/octet-stream,782367179,2023-08-03T19:32:20.569000Z,2023-08-03T19:43:29.150736Z,2023-08-03T19:43:39.634469Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/08/03/S2A_MSIL...,"[{'Value': '42e6de054c1e05e59e37edc3a370f921',...","{'Start': '2023-08-03T10:46:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-2.9366455 36.95...,"{'type': 'Polygon', 'coordinates': [[[-2.93664...",S2A_MSIL1C_20230803T104631_R051_T30SWG
3,application/octet-stream,02ef9b9b-94c6-4dab-8bec-45b822059c4c,S2A_MSIL1C_20230806T105621_N0509_R094_T30SVF_2...,application/octet-stream,797194252,2023-08-06T18:06:42.417000Z,2023-08-06T18:18:18.524711Z,2023-08-07T04:18:37.487578Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/08/06/S2A_MSIL...,"[{'Value': '03d929d7a08d47ea7ace4d16f2b42b89',...","{'Start': '2023-08-06T10:56:21.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-4.124939 37.041...,"{'type': 'Polygon', 'coordinates': [[[-4.12493...",S2A_MSIL1C_20230806T105621_R094_T30SVF
4,application/octet-stream,b5d7526b-a241-4027-ae23-c76a1f3f48a3,S2A_MSIL1C_20230806T105621_N0509_R094_T30SVG_2...,application/octet-stream,894248700,2023-08-06T17:59:00.349000Z,2023-08-06T18:05:17.230754Z,2023-08-06T18:05:28.593107Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/08/06/S2A_MSIL...,"[{'Value': 'a8a8df620e04ee493bf019d92b834e87',...","{'Start': '2023-08-06T10:56:21.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-4.1385193 37.94...,"{'type': 'Polygon', 'coordinates': [[[-4.13851...",S2A_MSIL1C_20230806T105621_R094_T30SVG
5,application/octet-stream,f386af74-2f88-4efa-bea7-4c776f7762ad,S2A_MSIL1C_20230806T105621_N0509_R094_T30SWF_2...,application/octet-stream,591684206,2023-08-06T17:58:50.163000Z,2023-08-06T18:07:25.676134Z,2023-08-06T18:07:33.885756Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/08/06/S2A_MSIL...,"[{'Value': '95c64cddf005f4af8083cdc4b54d0c74',...","{'Start': '2023-08-06T10:56:21.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-2.0238647 37.04...,"{'type': 'Polygon', 'coordinates': [[[-2.02386...",S2A_MSIL1C_20230806T105621_R094_T30SWF
6,application/octet-stream,3a263c3f-e32b-42e3-92ad-d13fad48625f,S2A_MSIL1C_20230806T105621_N0509_R094_T30SWG_2...,application/octet-stream,824218163,2023-08-06T18:07:40.842000Z,2023-08-06T18:16:04.375552Z,2023-08-06T18:16:16.145593Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/08/06/S2A_MSIL...,"[{'Value': '803826ae7f75f016e762204b95e71f58',...","{'Start': '2023-08-06T10:56:21.024000Z', 'End'...",geography'SRID=4326;POLYGON (

Find the corresponding dates

In [6]:
s2dates = [name.split('_')[2] for name in s2List['Name']]
s2dates = list(set(s2dates))
s2dates

['20230811T105629',
 '20230803T104631',
 '20230806T105621',
 '20230808T104629',
 '20230813T105031']

Loop through the different dates and bands

In [9]:
eps= 0.01 


for date in s2dates:

    date_obj = datetime.strptime(date, "%Y%m%dT%H%M%S").date()

    # get start and end date 
    start_date = date_obj.strftime("%Y-%m-%d") 
    end_date = (date_obj + timedelta(days=1))


    # load the collection
    s2_bands = eoconn.load_collection(
        "SENTINEL2_L1C",
        spatial_extent={'west':bbox[0]-eps,
                        'east':bbox[2]+eps,
                        'south':bbox[1]-eps,
                        'north':bbox[3]+eps,
                        'crs':4326},
        bands = bands,
        max_cloud_cover=cloud_prob,
        temporal_extent=[start_date, end_date],
    )

    if False:
        # resample to the target resolution
        s2_rsmpl = s2_bands.resample_spatial(resolution=res, 
                                     projection=crs,
                                     method = "cubic")

        # filter to the target extent
        s2_bbox = s2_rsmpl.filter_bbox(west=west, 
                                         south=south, 
                                         east=east, 
                                         north=north, 
                                         crs=crs)

    # convert from DN to toa
    #s2_toa = s2_bands.apply(lambda x: 0.0001 * x)   
    job = s2_bands.create_job(title="s2_" + date, 
                              out_format="GTiff")
    job.start()
    

    break



In [10]:
eoconn.list_jobs()

[{'created': '2023-12-22T16:52:01Z',
  'id': 'j-2312224eaa9148148e31b3bf88cd005d',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T16:54:19Z'},
 {'created': '2024-03-06T14:37:49Z',
  'id': 'j-240306e6f1b24087922604e79fb99043',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-03-06T14:40:12Z'},
 {'created': '2023-12-22T16:26:50Z',
  'id': 'j-231222b7bfd9430b987c26799ab64eab',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T16:29:20Z'},
 {'created': '2023-12-22T16:17:49Z',
  'id': 'j-231222292cc645b7973e650a7d99edfa',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T16:20:12Z'},
 {'created': '2024-03-06T14:40:21Z',
  'id': 'j-2403064687224c569a5eee8ecd0e5b76',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-03-06T14:42:27Z'},
 {'created': '2023-12-22T16:45:10Z',
  'id': 'j-231222943790487e9937862936910a16',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T16:47:23Z'},
 {'created': '2023-12-22T16:09:33Z',
  'id': 'j-23122289468f4d7b94cf9c14ff1b1c46',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T16:12:15Z'},
 {'created': '2024-03-06T14:42:50Z',
  'id': 'j-24030605a6e64241a858dc7bd4c41f2f',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-03-06T14:45:24Z'},
 {'created': '2023-12-22T16:22:18Z',
  'id': 'j-2312226fead84ec3aec5bd7d97b5e31e',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T16:24:13Z'},
 {'created': '2023-12-22T16:58:34Z',
  'id': 'j-2312225757ff41d3be1f962a949905ee',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T17:00:18Z'},
 {'created': '2024-06-07T13:31:10Z',
  'id': 'j-240607ae41684195a7ef4a1429e44ea8',
  'status': 'error',
  'updated': '2024-06-07T14:26:27Z'},
 {'created': '2024-06-07T13:38:15Z',
  'id': 'j-2406079ce2dc4a4e863f4e4881c3778f',
  'status': 'error',
  'updated': '2024-06-07T14:31:36Z'},
 {'created': '2024-06-07T13:26:01Z',
  'id': 'j-240607f0cca34a45a7d414dcca4699e6',
  'status': 'error',
  'updated': '2024-06-07T13:28:53Z'},
 {'created': '2024-06-07T11:37:44Z',
  'id': 'j-2406072110944ac9ae59a2fb48d47a10',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-06-07T12:12:57Z'},
 {'created': '2023-12-22T17:18:39Z',
  'id': 'j-231222c3c2a54f0489791163d46e892b',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T17:21:19Z'},
 {'created': '2023-12-22T16:55:30Z',
  'id': 'j-23122244395e4ad4bb4d1e12dd5b99e9',
  'progress': 100,
  'status': 'finished',
  'updated': '2023-12-22T16:57:13Z'},
 {'created': '2024-06-18T08:27:08Z',
  'id': 'j-240618fde5b049e189b93a32a10cafb6',
  'status': 'error',
  'updated': '2024-06-18T10:51:48Z'},
 {'created': '2024-07-23T08:15:52Z',
  'id': 'j-240723d976e94c90a6fb3d7629e18920',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-07-23T08:24:09Z'},
 {'created': '2024-07-22T11:42:42Z',
  'id': 'j-240722df032948d299da519f7c40a049',
  'status': 'error',
  'updated': '2024-07-22T11:44:15Z'},
 {'created': '2024-07-23T12:08:21Z',
  'id': 'j-240723affbba4af8838fc661b95f286e',
  'status': 'error',
  'updated': '2024-07-23T12:09:22Z'},
 {'created': '2024-07-29T09:38:07Z',
  'id': 'j-24072953229547e18f3d653113085330',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-07-29T09:45:11Z'},
 {'created': '2024-07-29T12:57:59Z',
  'id': 'j-2407294a4fd44b02b3b47a7c16566bb1',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-07-29T13:06:25Z'},
 {'created': '2024-07-29T12:32:49Z',
  'id': 'j-2407296a980e4203b87e424507ed48c9',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-07-29T12:44:19Z'},
 {'created': '2024-07-29T10:14:18Z',
  'id': 'j-24072938c43f4d5f896a6e54d8793433',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-07-29T10:24:15Z'},
 {'created': '2024-06-19T10:14:46Z',
  'id': 'j-240619920a98463c9f4b981144dc27e1',
  'status': 'error',
  'updated': '2024-06-19T10:17:07Z'},
 {'created': '2024-06-19T10:31:05Z',
  'id': 'j-240619

In [54]:
results = job.get_results()
job_id = job.job_id

for asset in results.get_assets():
    if asset.metadata["type"].startswith("image/tiff"):
        
        date = asset.name[7:17].replace('-','')
        
        # output file name
        s2_name = next((item.split('.')[0] for item in s2List['Name'] if date in item), None)
        tile = s2_name.split('_')[5]
        
        # create the output folder
        dirname = outdir + os.sep + s2_name.replace(tile,'merged')
        
        if not os.path.exists(dirname):
            print('Creating folder ' + os.path.basename(dirname))
            os.mkdir(dirname)
        
        # file name
        fname = dirname + os.sep + ('_').join(['merged', date, 'toa']) + '.tif'
        
        print('Downloading ' + os.path.basename(fname))
        asset.download(fname)
